In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

# model building
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Masking
from tensorflow.keras.metrics import RootMeanSquaredError
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN, Flatten
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dropout
from keras import layers
from tensorflow.keras import callbacks
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold

# URL Download
import csv
import io
import urllib.request
import requests

import joblib

2022-04-05 17:04:49.720505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-05 17:04:49.720543: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Functions

In [19]:
def load_data():
    """Read in the most recent bike theft data from Polizei Berlin and return
    a pandas dataframe """
    url = "https://www.internetwache-polizei-berlin.de/vdb/Fahrraddiebstahl.csv"
    download = requests.get(url)
    decoded_content = download.content.decode('ISO-8859-1')
    file = decoded_content.splitlines()

    cr = csv.DictReader(file, delimiter=',')
    my_list = list(cr)
    df  = pd.DataFrame(my_list)
    return df

In [20]:
def translate_col_names(d):
    eng_col_names = {
        "ANGELEGT_AM": "date_reported",
        "TATZEIT_ANFANG_DATUM": "date_theft_start",
        "TATZEIT_ANFANG_STUNDE": "hour_theft_start",
        "TATZEIT_ENDE_DATUM": "date_theft_end",
        "TATZEIT_ENDE_STUNDE": "hour_theft_end",
        "LOR": "LOR",
        "SCHADENSHOEHE": "estimated_value",
        "VERSUCH": "attempt",
        "ART_DES_FAHRRADS": "type_bike",
        "DELIKT": "theft_type",
        "ERFASSUNGSGRUND": "theft_type_detail"
    }
    d.rename(columns= eng_col_names, inplace=True)

In [21]:
# define function for renaming the categories
def rename_type_bike(x):
    """translation of the categories of variable "type_bike".
    """
    if x == "Herrenfahrrad":
        return "man's bike"
    if x == "Damenfahrrad":
        return "woman's bike"
    if x == "Fahrrad":
        return "bike"
    if x == "Kinderfahrrad":
        return "child's bike"
    else:
        return "other bike"

# dictionary for "attempt"
attempt_dict = {
    "Ja": "Yes",
    "Nein": "No",
    "Unbekannt": "Unknown"
}

In [22]:
# Concatenates translation of column and category names,
#  conversion of dtypes, drop duplicates and create
#  higher regional levels from LOR
def clean_theft_data(d):
    """Takes in the pd Dataframe created in load_data() and
    returns a clean dataframe"""
    #translate columns to English
    translate_col_names(d)

    #translate bike type to English
    d["type_bike"] = d["type_bike"].apply(rename_type_bike)

    #translate attempt type to English
    d["attempt"] = d["attempt"].map(attempt_dict)

    # convert the date columns to format='%d.%m.%Y
    d["date_reported"] = pd.to_datetime(d["date_reported"], format='%d.%m.%Y')
    d["date_theft_start"] = pd.to_datetime(d["date_theft_start"], format='%d.%m.%Y')
    d["date_theft_end"] = pd.to_datetime(d["date_theft_end"], format='%d.%m.%Y')

    # convert the time columns to int
    d["hour_theft_start"] = d["hour_theft_start"].astype(int)
    d["hour_theft_end"] = d["hour_theft_end"].astype(int)

    #convert value column to float
    d["estimated_value"] = d["estimated_value"].astype(float)

    #drop duplicates
    d = d.drop_duplicates()

    # BZR (first six numbers)
    d["BZR"] = d["LOR"].str[:6]

    # PGR (first four numbers)
    d["PGR"] = d["LOR"].str[:4]

    # Bezirk (first four numbers)
    d["Bezirk"] = d["LOR"].str[:2]

    return d

# Group by bezirk and sum up
def pivot_theft_data(d):
    """Groups dataframe by Bezirk and returns sum of thefts for
    each Bezirk and day (date_reported)"""
    d = d.pivot_table(index = "date_reported", columns = "Bezirk", values = "type_bike", aggfunc= "count")
    d.fillna(value = 0, inplace=True)
    return d


# Calculate percentage theft by Bezirk
def perc_split_bezirk(d):
    """returns df showing % split of bikes stolen over the last 2 weeks per Bezirk in Berlin"""
    d = d[-15:]
    d.loc['perc_split']= d.sum()
    d = d.div(d.sum(axis=1), axis=0)
    d = d.iloc[-1]
    return pd.DataFrame(d)


# Calculate rolling average
def calculate_rolling_average(df, window_size):
    """Calculate rolling average over the last window_size days.
    Fills missing values with mean of the last window_size days"""
    fill_value = df["total"][-window_size:].mean()
    df["total_moving_average"] = df["total"].rolling(window = window_size, center = False).mean().fillna(fill_value)

# Calculate the total number of reported stolen bikes in the last 365 days
def bikes_stolen_365():
    """returns total bikes reported stolen in the last 365 days in Berlin"""
    df = load_data()
    df = clean_theft_data(df)
    df = pivot_theft_data(df)
    df['Total'] = df.sum(axis=1)
    df = pd.DataFrame(df["Total"])
    df =df[-365:]
    total_stolen_365=df.sum().values[0]
    return int(total_stolen_365)

# Calculates "Every XX minutes a bike is reported as stolen in Berlin"
def theft_frequency():
    """returns frequency (in minutes) of bikes being reported as
    stolen in Berlin in the last 365 days"""
    minutes_day=1440
    minutes_year=1440*365
    return round(minutes_year/bikes_stolen_365())


# Create the dataframe for the modelling
def create_modelling_dataframe():
    """Read in most recent dataset from URL, clean it, group it
    and return dataframe for model creation
    """
    # load data
    df = load_data()
    # clean data
    df= clean_theft_data(df)
    # group data by Bezirk and date_reported and sum up
    df = pivot_theft_data(df)

    # add "total column"
    df["total"] = df.sum(axis = 1)

    # cut-off the last three days
    # df.drop(df.tail(3).index,inplace=True)

    # calculate rolling average
    calculate_rolling_average(df, window_size = 3)

    # select relevant columns for modelling
    cols_list =  ["total", "total_moving_average"]
    df = df[cols_list]

    return df

# calculates the mean estimated value of all reported stolen bikes
def mean_estimated_value():
    """Returns the mean of "estimated value" of all stolen bikes.
    "Kellereinbruch" is filtered out
    """
    df = load_data()
    df= clean_theft_data(df)
    start_date = datetime.datetime.today() - datetime.timedelta(365)
    cond = np.logical_and(df["theft_type"] != "Keller- und Bodeneinbruch", df["date_reported"] >= start_date)
    df = df[cond]
    return round(df.estimated_value.mean())


def hourly_count_stolen_bikes():
    """Creates a line plot of the number of stolen bikes by hour_theft_start
    """

    df = load_data()
    df = clean_theft_data(df)

    count_per_hour = df.groupby("hour_theft_start").count()
    count_per_hour = count_per_hour[["date_reported"]].reset_index()
    count_per_hour["count_stolen"] = count_per_hour["date_reported"]
    fig = px.line(count_per_hour, x='hour_theft_start', y='count_stolen',
                title='Berlin: Hourly count of stolen bikes from 2021-01-01',
                labels={"hour_theft_start": "Assumed hour of theft", "count_stolen": "Number of stolen bikes"})

    return fig

def get_last_date():
    df = load_data()
    df["date_reported"] = pd.to_datetime(df["ANGELEGT_AM"], format='%d.%m.%Y')
    last_date = df["date_reported"].max().date()
    return last_date


In [23]:
def create_opt_model():
    """
    Creates the finetuned model selected in B_Theft_Modelling
    Outputs models, that needs to be compiled and fit"""
    model = Sequential()
    # first LSTM layer
    model.add(LSTM(units = 70, activation = "tanh", return_sequences = True))
    model.add(Dropout(0.2))
    # second LSTM layer
    model.add(LSTM(units= 30, activation= "tanh", return_sequences= False))
    model.add(Dropout(0.2))

    # output layer to predict one value
    model.add(Dense(1, activation= "linear"))
    return model

def get_X_y(dataset, window_size, future_horizon):
    """Creates arrays to be fed into the RNN model
    Input: dataframe after create_modelling_dataframe, window_size and future_horizon
    Output: Arrays for X and y
    """
    X = []
    y = []

    for i in range(0, dataset.shape[0] - window_size - future_horizon):
        X.append(dataset[i: i + window_size])
        y.append(dataset["total"][i + window_size: i + window_size + future_horizon])

    X = np.array(X)
    y = np.array(y)

    # expand dimensions
    #X = np.expand_dims(X, 2)
    return X, y



# Read in data from URL

In [48]:
df = create_modelling_dataframe()

/tmp/ipykernel_1609/2723451935.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["BZR"] = d["LOR"].str[:6]
/tmp/ipykernel_1609/2723451935.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d["PGR"] = d["LOR"].str[:4]
/tmp/ipykernel_1609/2723451935.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexin

In [49]:
df.head()

Bezirk,total,total_moving_average
date_reported,,
2021-01-01,4.0,44.000000
2021-01-02,7.0,44.000000
2021-01-03,3.0,4.666667
2021-01-04,21.0,10.333333
2021-01-05,31.0,18.333333


# Train-test-split

In [50]:
len(df)

459

In [51]:
len_ = int(len(df)*0.8)

In [53]:
df_train = df[:len_]
df_test = df[len_:]

In [54]:
df_train.shape

(367, 2)

In [55]:
df_test.shape

(92, 2)

In [59]:
X_train, y_train = get_X_y(df_train, 31, 1)
X_test, y_test = get_X_y(df_test, 31, 1)

# Baseline model (last observation)

In [62]:
y_pred_base = X_test[:,-1,0]
y_pred_base.shape

(60,)

In [63]:
y_test.shape

(60, 1)

# Create empty model

In [41]:
model = create_opt_model()

2022-04-05 17:23:05.367109: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-05 17:23:05.367147: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-05 17:23:05.367164: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (LAPTOP-C0GM00LC): /proc/driver/nvidia/version does not exist
2022-04-05 17:23:05.367947: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [60]:
es = callbacks.EarlyStopping(patience = 5, restore_best_weights=True)

In [ ]:
if __name__ == "__main__":
    model = create_opt_model()
    model.compile(loss = "mse",
                  optimizer = "adam",
                  metrics = "mean_absolute_percentage_error")

    print(model.summary())